# Exercise 4 - Debugging and Testing

## General Notes


- All previous comments about naming files, tags and readable commit messages still apply.

- The code you write in this assignment will be the starting point for the next assignment. We therefore strongly recommend that you share the work equally, such that all students are familiar with the code. Also, groups who don't complete this assignment will have a very hard time during the next assignment.

- This time it is extremely important that you stick to the exact function interfaces (i.e. the name of the function and the name and order of all arguments). The reason is that we want to be able to run tests written by one group on functions written by another group. 



## Background

In this assignment you will continue to work towards a replication of Cunha, Heckman and Schennach (CHS), Econometrica, 2010. This time you will implement Kalman filters that can be used to make maximum likelihood estimation of Dynamic Latent factor models computationally feasible. Moreover, you will learn how to debug more efficiently and how to use Pytest to write unit tests for your functions. 

Kalman filters can be used for state and parameter estimation in so called state space models. They combine several measurements (in our case observed test scores) of an underlying state vector (in our case latent skills) in an optimal way. This even works if the state vector is not static (skills change from one period to the next). 

To make the relationship to the technology of skill formation and general state space models clearer, we can use the following notation:

Collect of latent factors in period $t$ in the vetcor $\mathbf{x}_t$:

$\mathbf{x}_t = (cognitive\_skills_t, \text{ } noncognitive\_skills_t, \text{ } investments_t, \text{ } parents\_cognitive\_skills_t, parents\_noncognitive\_skills_t)$

Collect all measurements in period $t$ in the vector $\mathbf{y}_t$

$\mathbf{y}_t = (IQ\_score_t, \text{ } math\_score_t, bpi\_antisocial_t, bpi\_anxiety_t, \ldots)$

Then the full model can be expressed by two fundamental equations:

$$\begin{align}
        \mathbf{x}_{t + 1} &= \mathbf{F}_t(\mathbf{x}_t) + \boldsymbol{\eta}_{t} &\quad{\text{transition equations}} \\
        \mathbf{y}_t &= \mathbf{H}_t \cdot \mathbf{x}_t + \boldsymbol{\epsilon}_t &\quad{\text{linear measurement equations}}
\end{align}$$

where $\mathbf{F}_t(\mathbf{x}_t)$ are the production functions of skills, $\mathbf{H}_t$ are the factor loadings that describe the relationship between observed measurements and latent skills, $\boldsymbol{\eta}_{t}$ is a vector of skill shocks and $\boldsymbol{\epsilon}_t$ is a vector of measurement errors.

You can find a detailed introduction to Kalman filters in `kalman_filters.pdf` in your repository. For assignment 4 you only need the description of the algorithms (highlighted by boxes). The rest is background information or might become relevant in future assignments. 

If you are a Macroeconomist you might fear (or hope) that you can reuse code from previous courses where you probably already implemented a Kalman filter. This will not be the case. Using Kalman filters for estimation purposes requires a numerically more stable version. Those algorithms were first derived for NASA's Apollo Missions and mankind would probably not have reached the moon without them.


Your main tasks will be to debug the predict step of the Kalman filter and to test and implement the update step. 

The *update step* takes an initial estimate of the state vector (skills) and refines it using a *measurement* (test scores).  It has the following interface.


``` python

def square_root_linear_update(state, root_cov, measurement, loadings):
    """Update *state* and *root_cov with* with a *measurement*.
    
    Args:
        state (pd.Series): pre-update estimate of the unobserved state vector
        root_cov (pd.DataFrame): lower triangular matrix square-root of the
            covariance matrix of the state vector before the update
        measurement (float): the measurement to incorporate
        loadings (pd.Series): the factor loadings
        
    Returns:
        updated_state (pd.Series)
        updated_root_cov (pd.Series)
    
    """

    return updated_state, updated_root_cov

```

The *predict step* takes the refined estimate of the state vector and uses the law of motion of the state vector (production functions of skills) to produce an estimate for the state vector in the next period. It has the following interface:

``` python 

def square_root_unscented_predict(state, root_cov, params, shock_sds, kappa):
    """Predict *state* in next period and adjust *root_cov*.

    Args:
        state (pd.Series): period t estimate of the unobserved state vector
        root_cov (pd.DataFrame): lower triangular matrix square-root of the
            covariance matrix of the state vector in period t
        params (dict): keys are the names of the states (latent
            factors), values are series with parameters for the transition
            equation of that state.
        shock_sds (pd.Series): standard deviations of the shocks
        kappa (float): scaling parameter for the unscented predict

    Returns:
        predicted_state (pd.Series)
        predicted_root_cov (pd.DataFrame)

    """

    return predicted_state, predicted_root_cov

```






## Tasks

1. Clone this repository, add a .gitignore-file as in previous assignments, and create branches with the github names of each group member. As last time, each of you will do work on his or her branch and occasionally merge the result into the master branch. This time you don't need a .tex file for the written solution.

2. Open a command line in the `code` folder and run all tests. So far, there are two tests in `test_predict.py`. Both will fail because the functions in `predict.py` contain bugs. 

3. Use a debugger to debug the "nice" bugs. By nice bugs we mean the ones that lead to exceptions and therefore cannot be overlooked. You are done with this step when the tests just fail because the functions produce the wrong numbers but not because they raise exceptions.

4. Write more unit tests to locate the "dangerous" bugs. By dangerous we mean the ones where the code runs through but produces wrong results. So far, `test_predict.py` does not contain unit tests in the strict sense, because we only tested `square_root_unscented_predict` and not the functions that are called inside that function. You are done when the two original tests pass.

    **Notes** 
    
    - Even if you happen to find all bugs just by looking at the code, we want you to write a test that would have located this bug as precisely as possible.

5. Write tests for `square_root_linear_update` in the module `test_update.py`. It is important that you write those tests before you implement the actual function! Commit when you are done but before you write the actual function. Set a tag called `tests_only`. We will check that you wrote the tests first in your commit history. 

    **Notes**
    - To generate test cases it is easier to use the formulae for the normal (i.e. non square-root) linear update. The resulting covariance matrix from the `square_root_linear_update` can easily be transformed to the full covariance matrix for testing.
    - Think carefully, when you design your test cases. It is often better to have a few well designed tests, than many tests that all cover the same cases. We will try to write a version of `square_root_linear_update` that contains a bug but passes your test. 
    - It might be helpful to use a coverage tool to see which parts of your code are covered by your tests. However, this is not mandatory.

6. Implement `square_root_linear_update` in `update.py`. Stick to the exact interface from above. Only start this after task five is completed. 

7. Make sure that all tests pass.

8. Once you are satisfied with your solution, merge it into the master branch of your repository. Then add a tag called `exercise_4`  and push all branches (including the master branch and the tag) to the central server.


